In [1]:
import pandas as pd
from functools import reduce


In [2]:
df_regiao_saude = pd.read_csv('./dataset/Região de Saúde (CIR).csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip').drop(columns="35121 Vale do Ribeira")
df_regiao_saude

,Municpio,Total
0,350925 CAJATI,2
1,350990 CANANEIA,3
2,351480 ELDORADO,6
3,352030 IGUAPE,13
4,352042 ILHA COMPRIDA,5
5,352330 ITARIRI,7
6,352460 JACUPIRANGA,2
7,352610 JUQUIA,11
8,352990 MIRACATU,9
9,353620 PARIQUERA-ACU,1828


In [3]:
df_adeq_quant = pd.read_csv('./dataset/Adeq quant pré-natal.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_anomalia = pd.read_csv('./dataset/Anomalia congênita.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_consult = pd.read_csv('./dataset/Consult pré-natal.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_duracao = pd.read_csv('./dataset/Duração gestação.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_estado_civil = pd.read_csv('./dataset/Estado civil mãe.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_idade_mae = pd.read_csv('./dataset/Idade da mãe.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_local = pd.read_csv('./dataset/Local ocorrência.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_ocorrencia_sexo = pd.read_csv('./dataset/ocorrênc por Sexo.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_peso_nascer = pd.read_csv('./dataset/Peso ao nascer.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')

df_tipo_gravidez = pd.read_csv('./dataset/Tipo de gravidez.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')
df_tipo_parto = pd.read_csv('./dataset/Tipo de parto.csv', encoding='ISO-8859-1', sep=';',on_bad_lines='skip')

# Criando uma lista com todos os DataFrames
dfs = [
df_adeq_quant,
df_anomalia,
df_consult,
df_duracao,
df_estado_civil,
df_idade_mae,
df_local,
df_ocorrencia_sexo,
df_peso_nascer,
df_tipo_gravidez,
df_tipo_parto
]

In [4]:
dfs = [df.drop(columns="Total", errors="ignore") for df in dfs]

for i, df in enumerate(dfs):

    df.replace('-', 0, inplace=True) # Convertendo todas as colunas (exceto a coluna 'Municpio') para numéricas

    cols = df.columns.drop('Municpio')

    df[cols] = df[cols].apply(pd.to_numeric)

    print(f"DataFrame {i+1}:")
    print(df.columns.tolist())
    print(df.head())  # Exibe as primeiras 5 linhas de cada DataFrame
    print(df.info())
    print(df.describe(include="all"))
    print(df.isna().sum())
    print("\n")  # Linha em branco para separar a saída de cada DataFrame



DataFrame 1:
['Municpio', 'qt adq Nao fez pre-natal', 'qt adq Inadequado', 'qt adq Intermediario', 'qt adq Adequado', 'qt adq Mais que adequado', 'qt adq Nao Classificados']
               Municpio  qt adq Nao fez pre-natal  qt adq Inadequado  \
0         350925 CAJATI                         0                  2   
1       350990 CANANEIA                         0                  2   
2       351480 ELDORADO                         0                  1   
3         352030 IGUAPE                         3                  4   
4  352042 ILHA COMPRIDA                         0                  3   

   qt adq Intermediario  qt adq Adequado  qt adq Mais que adequado  \
0                     0                0                         0   
1                     0                0                         1   
2                     0                1                         3   
3                     1                2                         3   
4                     1                0   

In [5]:
df_final = reduce(lambda left, right: pd.merge(left, right, on='Municpio', how='outer'), dfs)
df_final = pd.merge(df_final, df_regiao_saude, on='Municpio', how='outer')
# Exibir o DataFrame final
print(df_final.head())
print(df_final.info())

          Municpio  qt adq Nao fez pre-natal  qt adq Inadequado  \
0                                        NaN                NaN   
1    350925 CAJATI                       0.0                2.0   
2  350990 CANANEIA                       0.0                2.0   
3  351480 ELDORADO                       0.0                1.0   
4    352030 IGUAPE                       3.0                4.0   

   qt adq Intermediario  qt adq Adequado  qt adq Mais que adequado  \
0                   NaN              NaN                       NaN   
1                   0.0              0.0                       0.0   
2                   0.0              0.0                       1.0   
3                   0.0              1.0                       3.0   
4                   1.0              2.0                       3.0   

   qt adq Nao Classificados  anomalia Sim  anomalia Nao  anomalia Ignorado  \
0                       NaN           NaN           NaN                NaN   
1                   

In [6]:
df_final.to_csv('base_final.csv', index=False)

In [7]:
df_final.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

# Preencher NaN com 0
df_final.fillna(0, inplace=True)

# Converter colunas numéricas para inteiro
for col in df_final.select_dtypes(include=['float']).columns:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0).astype(int)

# Exibir o DataFrame final
print(df_final.head())


          Municpio  qt adq Nao fez pre-natal  qt adq Inadequado  \
0                0                         0                  0   
1    350925 CAJATI                         0                  2   
2  350990 CANANEIA                         0                  2   
3  351480 ELDORADO                         0                  1   
4    352030 IGUAPE                         3                  4   

   qt adq Intermediario  qt adq Adequado  qt adq Mais que adequado  \
0                     0                0                         0   
1                     0                0                         0   
2                     0                0                         1   
3                     0                1                         3   
4                     1                2                         3   

   qt adq Nao Classificados  anomalia Sim  anomalia Nao  anomalia Ignorado  \
0                         0             0             0                  0   
1                   

In [8]:
df_final.to_csv('base_final.csv', index=False)

In [10]:
pd.read_csv('base_final.csv')

,Municpio,qt adq Nao fez pre-natal,qt adq Inadequado,qt adq Intermediario,qt adq Adequado,qt adq Mais que adequado,qt adq Nao Classificados,anomalia Sim,anomalia Nao,anomalia Ignorado,...,peso 1500 a 2499 g,peso 2500 a 2999 g,peso 3000 a 3999 g,peso 4000g e mais,peso Ignorado,gestacao unica,gestacao Dupla,tp de parto Vaginal,tp de parto Cesario,Total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,350925 CAJATI,0,2,0,0,0,0,0,2,0,...,0,1,1,0,0,2,0,2,0,2
2,350990 CANANEIA,0,2,0,0,1,0,0,3,0,...,0,0,3,0,0,3,0,3,0,3
3,351480 ELDORADO,0,1,0,1,3,1,0,5,1,...,1,2,3,0,0,6,0,6,0,6
4,352030 IGUAPE,3,4,1,2,3,0,0,13,0,...,2,7,4,0,0,13,0,13,0,13
5,352042 ILHA COMPRIDA,0,3,1,0,0,1,0,5,0,...,1,1,3,0,0,5,0,4,1,5
6,352330 ITARIRI,1,3,0,0,3,0,0,7,0,...,1,3,3,0,0,7,0,7,0,7
7,352460 JACUPIRANGA,0,0,0,0,0,2,0,1,1,...,0,1,1,0,0,2,0,2,0,2
8,352610 JUQUIA,0,0,1,1,6,3,0,11,0,...,3,3,3,0,0,9,2,11,0,11
9,352990 MIRACATU,0,2,1,2,3,1,0,7,2,...,2,1,5,1,0,9,0,9,0,9


In [11]:
df_final.head()

,Municpio,qt adq Nao fez pre-natal,qt adq Inadequado,qt adq Intermediario,qt adq Adequado,qt adq Mais que adequado,qt adq Nao Classificados,anomalia Sim,anomalia Nao,anomalia Ignorado,...,peso 1500 a 2499 g,peso 2500 a 2999 g,peso 3000 a 3999 g,peso 4000g e mais,peso Ignorado,gestacao unica,gestacao Dupla,tp de parto Vaginal,tp de parto Cesario,Total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,350925 CAJATI,0,2,0,0,0,0,0,2,0,...,0,1,1,0,0,2,0,2,0,2
2,350990 CANANEIA,0,2,0,0,1,0,0,3,0,...,0,0,3,0,0,3,0,3,0,3
3,351480 ELDORADO,0,1,0,1,3,1,0,5,1,...,1,2,3,0,0,6,0,6,0,6
4,352030 IGUAPE,3,4,1,2,3,0,0,13,0,...,2,7,4,0,0,13,0,13,0,13


In [12]:
df_final.describe(include="all")

,Municpio,qt adq Nao fez pre-natal,qt adq Inadequado,qt adq Intermediario,qt adq Adequado,qt adq Mais que adequado,qt adq Nao Classificados,anomalia Sim,anomalia Nao,anomalia Ignorado,...,peso 1500 a 2499 g,peso 2500 a 2999 g,peso 3000 a 3999 g,peso 4000g e mais,peso Ignorado,gestacao unica,gestacao Dupla,tp de parto Vaginal,tp de parto Cesario,Total
count,15.0,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
unique,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.733333,65.333333,24.666667,28.000000,344.933333,5.733333,3.066667,466.666667,0.666667,...,25.733333,92.666667,310.666667,36.266667,0.133333,460.400000,10.000000,313.600000,156.800000,470.400000
std,NaN,3.534860,144.004795,54.400193,62.178315,766.304254,11.828939,6.829209,1032.400093,1.345185,...,58.633811,204.029293,688.213803,80.692066,0.351866,1018.010932,23.111531,694.519855,350.827634,1040.243982
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,2.500000,0.000000,2.500000,0.000000,2.500000
50%,NaN,0.000000,2.000000,1.000000,0.000000,3.000000,1.000000,0.000000,5.000000,0.000000,...,1.000000,1.000000,3.000000,0.000000,0.000000,6.000000,0.000000,6.000000,0.000000,6.000000
75%,NaN,2.000000,3.500000,1.000000,2.000000,4.500000,2.500000,0.000000,12.000000,1.000000,...,2.500000,5.000000,4.500000,0.500000,0.000000,11.000000,1.000000,12.000000,0.500000,12.000000


In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Municpio                          15 non-null     object
 1   qt adq Nao fez pre-natal          15 non-null     int64 
 2   qt adq Inadequado                 15 non-null     int64 
 3   qt adq Intermediario              15 non-null     int64 
 4   qt adq Adequado                   15 non-null     int64 
 5   qt adq Mais que adequado          15 non-null     int64 
 6   qt adq Nao Classificados          15 non-null     int64 
 7   anomalia Sim                      15 non-null     int64 
 8   anomalia Nao                      15 non-null     int64 
 9   anomalia Ignorado                 15 non-null     int64 
 10  consulta pre Nenhuma              15 non-null     int64 
 11  consulta pre De 1 a 3 consultas   15 non-null     int64 
 12  consulta pre De 4 a 6 co